# Let's filter Mixture-of-Thoughts 'default' split down to 4096 toks

In [4]:
from datasets import load_dataset
mot_all = load_dataset("open-r1/Mixture-of-Thoughts", "all")['train']

In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")
tokenizer.chat_template

'{{- bos_token }}\n{%- if custom_tools is defined %}\n    {%- set tools = custom_tools %}\n{%- endif %}\n{%- if not tools_in_user_message is defined %}\n    {%- set tools_in_user_message = true %}\n{%- endif %}\n{%- if not date_string is defined %}\n    {%- set date_string = "26 Jul 2024" %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n\n{#- This block extracts the system message, so we can slot it into the right place. #}\n{%- if messages[0][\'role\'] == \'system\' %}\n    {%- set system_message = messages[0][\'content\']|trim %}\n    {%- set messages = messages[1:] %}\n{%- else %}\n    {%- set system_message = "" %}\n{%- endif %}\n\n{#- System message + builtin tools #}\n{{- "<|start_header_id|>system<|end_header_id|>\\n\\n" }}\n{%- if builtin_tools is defined or tools is not none %}\n    {{- "Environment: ipython\\n" }}\n{%- endif %}\n{%- if builtin_tools is defined %}\n    {{- "Tools: " + builtin_tools | reject(\'equalto\', \'code_interp

In [3]:
tokenizer.chat_template = "{{- bos_token }}\n{%- if custom_tools is defined %}\n    {%- set tools = custom_tools %}\n{%- endif %}\n{%- if not tools_in_user_message is defined %}\n    {%- set tools_in_user_message = true %}\n{%- endif %}\n{%- if not date_string is defined %}\n    {%- set date_string = \"26 Jul 2024\" %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n\n{#- This block extracts the system message, so we can slot it into the right place. #}\n{%- if messages[0]['role'] == 'system' %}\n    {%- set system_message = messages[0]['content']|trim %}\n    {%- set messages = messages[1:] %}\n{%- else %}\n    {%- set system_message = \"You are a helpful AI Assistant that provides well-reasoned and detailed responses. You first think about the reasoning process as an internal monologue and then provide the user with the answer. Respond in the following format: <think>\n...\n</think>\n<answer>\n...\n</answer>\" %}\n{%- endif %}\n\n{#- System message + builtin tools #}\n{{- \"<|start_header_id|>system<|end_header_id|>\\n\\n\" }}\n{%- if builtin_tools is defined or tools is not none %}\n    {{- \"Environment: ipython\\n\" }}\n{%- endif %}\n{%- if builtin_tools is defined %}\n    {{- \"Tools: \" + builtin_tools | reject('equalto', 'code_interpreter') | join(\", \") + \"\\n\\n\"}}\n{%- endif %}\n{{- \"Cutting Knowledge Date: December 2023\\n\" }}\n{{- \"Today Date: \" + date_string + \"\\n\\n\" }}\n{%- if tools is not none and not tools_in_user_message %}\n    {{- \"You have access to the following functions. To call a function, please respond with JSON for a function call.\" }}\n    {{- 'Respond in the format {\"name\": function name, \"parameters\": dictionary of argument name and its value}.' }}\n    {{- \"Do not use variables.\\n\\n\" }}\n    {%- for t in tools %}\n        {{- t | tojson(indent=4) }}\n        {{- \"\\n\\n\" }}\n    {%- endfor %}\n{%- endif %}\n{{- system_message }}\n{{- \"<|eot_id|>\" }}\n\n{#- Custom tools are passed in a user message with some extra guidance #}\n{%- if tools_in_user_message and not tools is none %}\n    {#- Extract the first user message so we can plug it in here #}\n    {%- if messages | length != 0 %}\n        {%- set first_user_message = messages[0]['content']|trim %}\n        {%- set messages = messages[1:] %}\n    {%- else %}\n        {{- raise_exception(\"Cannot put tools in the first user message when there's no first user message!\") }}\n    {%- endif %}\n    {{- '<|start_header_id|>user<|end_header_id|>\\n\\n' -}}\n    {{- \"Given the following functions, please respond with a JSON for a function call \" }}\n    {{- \"with its proper arguments that best answers the given prompt.\\n\\n\" }}\n    {{- 'Respond in the format {\"name\": function name, \"parameters\": dictionary of argument name and its value}.' }}\n    {{- \"Do not use variables.\\n\\n\" }}\n    {%- for t in tools %}\n        {{- t | tojson(indent=4) }}\n        {{- \"\\n\\n\" }}\n    {%- endfor %}\n    {{- first_user_message + \"<|eot_id|>\"}}\n{%- endif %}\n\n{%- for message in messages %}\n    {%- if not (message.role == 'ipython' or message.role == 'tool' or 'tool_calls' in message) %}\n        {%- if message['role'] == 'assistant' %}\n            {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' }}\n            {% generation %}\n            {{- message['content'] | trim + '<|eot_id|>' }}\n            {% endgeneration %}\n        {%- else %}\n            {{- '<|start_header_id|>' + message['role'] + '<|end_header_id|>\\n\\n'+ message['content'] | trim + '<|eot_id|>' }}\n        {%- endif %}\n    {%- elif 'tool_calls' in message %}\n        {%- if not message.tool_calls|length == 1 %}\n            {{- raise_exception(\"This model only supports single tool-calls at once!\") }}\n        {%- endif %}\n        {%- set tool_call = message.tool_calls[0].function %}\n        {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' -}}\n        {% generation %}\n        {%- if builtin_tools is defined and tool_call.name in builtin_tools %}\n            {{- \"<|python_tag|>\" + tool_call.name + \".call(\" }}\n            {%- for arg_name, arg_val in tool_call.arguments | items %}\n                {{- arg_name + '=\"' + arg_val + '\"' }}\n                {%- if not loop.last %}\n                    {{- \", \" }}\n                {%- endif %}\n            {%- endfor %}\n            {{- \")\" }}\n        {%- else %}\n            {{- '{\"name\": \"' + tool_call.name + '\", ' }}\n            {{- '\"parameters\": ' }}\n            {{- tool_call.arguments | tojson }}\n            {{- \"}\" }}\n        {%- endif %}\n        {%- if builtin_tools is defined %}\n            {{- \"<|eom_id|>\" }}\n        {%- else %}\n            {{- \"<|eot_id|>\" }}\n        {%- endif %}\n        {% endgeneration %}\n    {%- elif message.role == \"tool\" or message.role == \"ipython\" %}\n        {{- \"<|start_header_id|>ipython<|end_header_id|>\\n\\n\" }}\n        {%- if message.content is mapping or message.content is iterable %}\n            {{- message.content | tojson }}\n        {%- else %}\n            {{- message.content }}\n        {%- endif %}\n        {{- \"<|eot_id|>\" }}\n    {%- endif %}\n{%- endfor %}\n{%- if add_generation_prompt %}\n    {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' }}\n{%- endif %}" #adapted for Llama with reasoning chat template, and generation tags

In [5]:
mot_all

Dataset({
    features: ['messages', 'num_tokens', 'source'],
    num_rows: 349317
})

In [6]:
mot_first = mot_all[0]
return_dict = tokenizer.apply_chat_template(
    mot_first['messages'],
    return_dict=True,
    return_assistant_tokens_mask=True,
)
length = len(return_dict['input_ids'])

for i in range(length):
    input_id = return_dict['input_ids'][i]
    decoded = tokenizer.decode([input_id])
    attention_mask = return_dict['attention_mask'][i]
    assistant_masks = return_dict['assistant_masks'][i]
    print(i, attention_mask, assistant_masks, input_id, repr(decoded))

0 1 0 128000 '<|begin_of_text|>'
1 1 0 128006 '<|start_header_id|>'
2 1 0 9125 'system'
3 1 0 128007 '<|end_header_id|>'
4 1 0 271 '\n\n'
5 1 0 38766 'Cut'
6 1 0 1303 'ting'
7 1 0 33025 ' Knowledge'
8 1 0 2696 ' Date'
9 1 0 25 ':'
10 1 0 6790 ' December'
11 1 0 220 ' '
12 1 0 2366 '202'
13 1 0 18 '3'
14 1 0 198 '\n'
15 1 0 15724 'Today'
16 1 0 2696 ' Date'
17 1 0 25 ':'
18 1 0 220 ' '
19 1 0 1627 '26'
20 1 0 10263 ' Jul'
21 1 0 220 ' '
22 1 0 2366 '202'
23 1 0 19 '4'
24 1 0 271 '\n\n'
25 1 0 2675 'You'
26 1 0 527 ' are'
27 1 0 264 ' a'
28 1 0 11190 ' helpful'
29 1 0 15592 ' AI'
30 1 0 22103 ' Assistant'
31 1 0 430 ' that'
32 1 0 5825 ' provides'
33 1 0 1664 ' well'
34 1 0 5621 '-re'
35 1 0 1525 'ason'
36 1 0 291 'ed'
37 1 0 323 ' and'
38 1 0 11944 ' detailed'
39 1 0 14847 ' responses'
40 1 0 13 '.'
41 1 0 1472 ' You'
42 1 0 1176 ' first'
43 1 0 1781 ' think'
44 1 0 922 ' about'
45 1 0 279 ' the'
46 1 0 33811 ' reasoning'
47 1 0 1920 ' process'
48 1 0 439 ' as'
49 1 0 459 ' an'
50 1 0 5

In [7]:
full_length_mot = 172514
lengths = []
from tqdm.notebook import tqdm
for i in tqdm(range(1000)):
    item = mot_all[i]
    return_dict = tokenizer.apply_chat_template(
        item['messages'],
        return_dict=True,
        return_assistant_tokens_mask=True,
    )
    length = len(return_dict['input_ids'])
    lengths.append(length)

mean = sum(lengths) / len(lengths)
maximum = max(lengths)
print(mean)
print(maximum)

  0%|          | 0/1000 [00:00<?, ?it/s]

5605.438
25343


In [8]:
# check tokenized length
max_length = 4096
def filter_long_rows(example):
    messages = example['messages']
    tokenized = tokenizer.apply_chat_template(messages, tokenize=True)
    length = len(tokenized)
    return length <= max_length

In [9]:
mot_filtered = mot_all.filter(filter_long_rows, num_proc=16)

Filter (num_proc=16):   0%|          | 0/349317 [00:00<?, ? examples/s]

In [10]:
mot_filtered.push_to_hub("Neelectric/MoT_all_Llama3_4096toks")

Uploading the dataset shards:   0%|          | 0/8 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

...

New Data Upload: |          |  0.00B /  0.00B            

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/datasets/Neelectric/MoT_all_Llama3_4096toks/commit/197e10065b9c7e02d4b8518d1b49774dbcfcb937', commit_message='Upload dataset', commit_description='', oid='197e10065b9c7e02d4b8518d1b49774dbcfcb937', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Neelectric/MoT_all_Llama3_4096toks', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Neelectric/MoT_all_Llama3_4096toks'), pr_revision=None, pr_num=None)

In [12]:
# check tokenized length
max_length = 8192
def filter_long_rows(example):
    messages = example['messages']
    tokenized = tokenizer.apply_chat_template(messages, tokenize=True)
    length = len(tokenized)
    return length <= max_length
science_filtered_longer = mot_all.filter(filter_long_rows, num_proc=16)
science_filtered_longer.push_to_hub("Neelectric/MoT_all_Llama3_8192toks")

Filter (num_proc=16):   0%|          | 0/349317 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/11 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

...

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

README.md:   0%|          | 0.00/430 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Neelectric/MoT_all_Llama3_8192toks/commit/952bd29a0eaa148d404ec9bcff47f135603808f2', commit_message='Upload dataset', commit_description='', oid='952bd29a0eaa148d404ec9bcff47f135603808f2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Neelectric/MoT_all_Llama3_8192toks', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Neelectric/MoT_all_Llama3_8192toks'), pr_revision=None, pr_num=None)